In [1]:
import threading
import pandas as pd
import csv
import time
import re
import os

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

In [2]:
class SharedListURLs:
    def __init__(self):
        self.allURLs = ["https://aujourdhui.ma/contact",
                        "https://lematin.ma/contact",
                        "https://www.goud.ma/%d9%84%d9%84%d8%a5%d8%aa%d8%b5%d8%a7%d9%84/",
                        "https://lefaso.net/spip.php?article64819",
                        "https://menara.ma/qui-sommes-nous",
                        "https://www.leconomiste.com/contact",
                        "https://www.ecomedias.ma/",
                        "https://www.alalam.ma/%D8%A7%D8%AA%D8%B5%D9%84-%D8%A8%D9%86%D8%A7_a8374.html?preview=1",
                        "https://www.alalam.ma/%D9%84%D9%84%D8%A5%D8%B4%D9%87%D8%A7%D8%B1_a8373.html?preview=1",
                        "https://www.lavieeco.com/nous-contacter/",
                        "https://femmesdumaroc.com/",
                        "https://aujourdhui.ma/qui-sommes-nous",
                        "https://lobservateur.info/contact",
                        "https://www.libe.ma/pages/",
                        "https://kifache.com/%d8%a7%d8%aa%d8%b5%d9%84-%d8%a8%d9%86%d8%a7/",
                        "https://kifache.com/%d9%84%d9%84%d8%a5%d8%b4%d9%87%d8%a7%d8%b1/",
                        "https://kifache.com/%d9%87%d9%8a%d8%a3%d8%a9-%d8%a7%d9%84%d8%aa%d8%ad%d8%b1%d9%8a%d8%b1/",
                        "https://medi1tv.com/ar/contact",
                        "https://medi1tv.com/ar/regie",
                        "https://medi1tv.com/ar/chaine",
                        "https://www.hitradio.ma/contact",
                        "https://www.neoregie.ma/",
                        "https://weblogy.com/nous-contacter",
                        "https://www.hibasport.com/contact.html",
                        "https://www.meretmarine.com/fr/nous-contacter#no-back",
                        "https://www.infomediaire.net/contact/",
                        "https://www.lemag.ma/contacter-lequipe/",
                        "https://pub.yabiladi.com/?_gl=1*lsanog*_ga*MTMxMDgzMjYzMC4xNzE1ODU4OTcw*_ga_03H215L3M0*MTcxNTg1ODk2OS4xLjEuMTcxNTg1OTAyOS4wLjAuMA..",
                        "https://www.alarabiya.net/tools/contact-us",
                        "https://burkina24.com/contactez-nous/",
                        "https://burkina24.com/Publicite-burkina24/",
                        "https://lareleve.ma/contactez-nous/",
                        "https://febrayer.com/%d8%a7%d9%84%d8%b1%d8%a6%d9%8a%d8%b3%d9%8a%d8%a9/about-us",
                        "https://www.akhbarona.com/contact/",
                        "https://www.akhbarona.com/team.html",
                        "https://www.educarriere.ci/",
                        "https://www.elkhabar.com/press/article/162824/%D8%A7%D8%AA%D8%B5%D9%84-%D8%A8%D9%86%D8%A7/",
                        "https://www.elkhabar.com/press/article/162825/%D8%A7%D9%84%D8%A5%D8%B4%D9%87%D8%A7%D8%B1-%D8%B9%D9%84%D9%89-%D8%A7%D9%84%D9%85%D9%88%D9%82%D8%B9/",
                        "https://aide.lesechos.fr/#faq-formulaire#utm_source=le%3Alec0f&utm_medium=click&utm_campaign=footer_help",
                        "https://medias.lesechosleparisien.fr/portfolio/accueil/#utm_source=le%3Alec0f&utm_medium=click&utm_campaign=footer_ad",
                        "https://kech24.com/%d9%81%d8%b1%d9%8a%d9%82-%d8%a7%d9%84%d8%b9%d9%85%d9%84.html",
                        "https://www.marrakech24.info/%d8%a5%d8%aa%d8%b5%d9%84-%d8%a8%d9%86%d8%a7",
                        "https://www.rewmi.com/",
                        "https://www.koaci.com/contact.html",
                        "https://www.sahara24.net/team/",
                        "https://www.zoodomail.com/",
                        "https://www.20minutes-media.com/contacts",
                        "https://www.20minutes.fr/contact",
                        "https://www.usinenouvelle.com/contact",
                        "https://www.jeuneafrique.com/emploi-formation/recruteurs/646634/jeune-afrique-media-groupe/",
                        "https://www.jeuneafrique.com/contacts/",
                        "https://www.agenceecofin.com/",
                        "https://www.xalimasn.com/contacts/",
                        "https://rue20.com/%d8%a5%d8%aa%d8%b5%d9%84-%d8%a8%d9%86%d8%a7",
                        "https://rue20.com/%d8%a3%d8%b9%d9%84%d9%86-%d9%85%d8%b9%d9%86%d8%a7",
                        "https://tanja24.com/contact-2/",
                        "https://presstetouan.com/contact/",
                        "https://presstetouan.com/%d9%87%d9%8a%d8%a6%d8%a9-%d8%a7%d9%84%d8%aa%d8%ad%d8%b1%d9%8a%d8%b1/",
                        "https://zaiocity.net/contact/",
                        "https://fesnews.net/contactus/",
                        "https://fesnews.net/%d8%a7%d8%b9%d9%84%d8%a7%d9%86%d8%a7%d8%aa/",
                        "https://article19.ma/accueil/contact",
                        "https://www.cawalisse.com/%d9%84%d9%84%d8%a7%d8%aa%d8%b5%d8%a7%d9%84-%d8%a8%d9%86%d8%a7/",
                        "https://www.ariffino.net/%d8%a5%d8%aa%d8%b5%d9%84-%d8%a8%d9%86%d8%a7",
                        "https://klamkom.com/%d9%84%d9%84%d8%a5%d8%aa%d8%b5%d8%a7%d9%84-%d8%a8%d9%86%d8%a7/",
                        "https://fr.le360.ma/contact/",
                        "https://fr.le360.ma/publicit%C3%A9/",
                        "https://hisspress.net/%d8%a7%d8%aa%d8%b5%d9%84-%d8%a8%d9%86%d8%a7/",
                        "https://www.financialafrik.com/contacts/",
                        "https://www.assahraoui.info/contact",
                        "https://www.assahraoui.info/info",
                        #########
                        "https://www.safitoday.com/%d9%85%d9%86-%d9%86%d8%ad%d9%86/",
                        "https://www.mapexpress.ma/contact/",
                        "https://mapdigitale.ma/contactez-map-digitale/",
                        "https://www.arabnews.com/contact-us",
                        "https://www.bakersfield.com/site/about.html",
                        "https://www.bakersfield.com/site/contact.html",
                        "https://www.mercurynews.com/contact-us/",
                        "https://www.businessdailyafrica.com/bd/meta/contact-us-4584254",
                        "https://www.businessghana.com/site/contact_us",
                        "https://www.business-standard.com/contact-us",
                        "https://help.usatoday.com/contact-us",
                        "https://www.gannett.com/brands/#usat",
                        "https://www.eagletribune.com/contact_us/",
                        "https://myaccount.economist.com/s/contact-us",
                        "https://www.emirates247.com/contact-us-1.131",
                        "https://www.emirates247.com/advertise-with-us-1.635432",
                        "https://www.euronews.com/contact",
                        "https://www.express.co.uk/about-us",
                        "https://www.flightglobal.com/contact-us/",
                        "https://www.francetvpub.fr/marques/france-24/",
                        "https://www.ghanabusinessnews.com/about-us/",
                        "https://gulfnews.com/contact-us",
                        "https://www.heraldscotland.com/contact/",
                        "https://www.koreatimes.co.kr/www2/common/contactus.asp",
                        "https://menafn.com/mf_contact.aspx",
                        "https://middle-east-online.com/contact",
                        "https://www.miningweekly.com/page/about-us",
                        "https://www.mirror.co.uk/contact-us/",
                        "https://www.monitor.co.ug/uganda/contact-us-1934576",
                        "https://www.nation.com.pk/contact-us"
                        
                        
                       ]
        self.data = []
        self.lock = threading.Lock()
        self.nb_URLs=len(self.allURLs)
        
    def load_data(self) :
        file_xslx_name = "./data/WebSites.xlsx"

        def url_processing(url):
            if url.startswith("http//"):
                return url.replace("http//", "http://")
            elif url.startswith("https//"):
                return url.replace("https//", "https://")
            elif not url.startswith("http://") and not url.startswith("https://"):
                return "https://" + url
            else:
                return url

        df = pd.read_excel(file_xslx_name)
        
        df['url'] = df['domain'].apply(url_processing)

        self.allURLs = df['url'].tolist()
        self.nb_URLs = len(self.allURLs)
        
    def select_data(self, start, end):
        if end < self.nb_URLs:
            self.data = self.allURLs[start:end]
        else:
            self.data = self.allURLs[start:self.nb_URLs]

In [3]:
import csv
import threading

class SentenceStorage:
    def __init__(self):
        self.lock_file = threading.Lock()
    
    def save_sentences_csv(self, sentences, csv_filename="sentences_from_web_2.csv") :
        header = ["id", "sentences"]
    
        file_exists = os.path.exists(csv_filename)
        is_empty = os.path.getsize(csv_filename) == 0 if file_exists else True
    
        last_id = 0
        if file_exists and not is_empty:
            with open(csv_filename, 'r', newline='', encoding='utf-8-sig') as csvfile:
                csv_reader = csv.reader(csvfile)
                rows = list(csv_reader)
                if len(rows) > 1:
                    last_row = rows[-1]
                    last_id = int(last_row[0])
    
        with open(csv_filename, 'a', newline='', encoding='utf-8-sig') as csvfile:
            csv_writer = csv.writer(csvfile)
    
            if is_empty:
                csv_writer.writerow(header)
    
            current_id = last_id
            for sentence in sentences:
                current_id += 1
                csv_writer.writerow([current_id, sentence])

In [4]:
class SentenceProcessing:

    def extract_chunks(self, contenu_html, max_len_chunk = 400, pad = 6):
        soup = BeautifulSoup(contenu_html, 'html.parser')
        text = str(soup.text)
        new_text = ' '.join([n.strip() for n in text.split('\n') if n != ''])
        text_list = new_text.split()
        len_text_list = len(text_list)
        n_chunks= int(len_text_list / max_len_chunk)+1
        chunks = []
        for i in range(n_chunks):
            if i == 0:
                start = i*max_len_chunk
                end = min((i+1)*max_len_chunk,len_text_list)
            else:
                start = end - pad
                end = min(((i+1)*max_len_chunk) - pad , len_text_list)
                
            chunk = text_list[start:end]
            chunks.append(' '.join(chunk))
            
            if i == n_chunks-1 and end != len_text_list and len(chunk)==max_len_chunk:
                start = end - pad
                end = len_text_list
                chunk = text_list[start:end]
                chunks.append(' '.join(chunk))
        return chunks

In [5]:
class WorkerThread(threading.Thread):
    
    def __init__(self, thread_id, shared_list, sentence_storage):
        super(WorkerThread, self).__init__()
        self.thread_id = thread_id
        self.shared_list = shared_list
        self.sentence_storage = sentence_storage
        
        # IP address and port and server of the Selenium hub and browser options
        self.HUB_HOST = "192.168.3.32"
        self.HUB_PORT = 4444
        self.server = f"http://{self.HUB_HOST}:{self.HUB_PORT}/wd/hub"
        self.options = webdriver.ChromeOptions()
        self.driver = None
        self.sentenceProcessing = SentenceProcessing()
        
    def run(self):
        while True:
            try:
                # Access the shared list synchronously
                with self.shared_list.lock:
                    if not self.shared_list.data:
                        break  # The list is empty, the thread ends

                    url = self.shared_list.data.pop(0)

                    print(f"%"*50)
                    print(f"url_item = {url} |||| by_thread_id = {self.thread_id}")
                    print(f"%"*50)
                
                self.driver = webdriver.Remote(command_executor=self.server, options=self.options)
                self.scraping_sentences(url)

            except IndexError:
                pass  # La liste était probablement vide, pas besoin de gérer l'exception ici ???????????????????


    def scraping_sentences(self, _url):
        try:
            self.driver.get(_url)
            WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.TAG_NAME, "body"))
             )
            _url = self.driver.current_url
            contenu_html = self.driver.page_source
            sentences = self.sentenceProcessing.extract_chunks(str(contenu_html), max_len_chunk = 300)
            with self.sentence_storage.lock_file:
                self.sentence_storage.save_sentences_csv(sentences)

        except Exception as e:
            error_message = f"Error :\n==>>>(in: scraping_sentences() at WorkerThread)\n==>>> {e.args[0]}"
            print(error_message)
            
        finally:
            self.driver.quit()


In [6]:
################################################ main ###########################################################
#################################################################################################################
# https://www.almarrakchia.net/#google_vignette

# os.makedirs("results", exist_ok=True)

# Créer une liste partagée, une file de résultats et un verrou CSV
shared_list = SharedListURLs()
# shared_list.load_data()

# Créer et démarrer les threads
num_threads = 30

start = 0
end = min(1000,shared_list.nb_URLs)
step = shared_list.nb_URLs

for i in range(start, end, step):
    
    shared_list.select_data(i,i+step)
    
    sentence_storage = SentenceStorage()
    
    threads = []
    for j in range(num_threads):
        thread = WorkerThread(j+1, shared_list, sentence_storage)
        thread.start()
        threads.append(thread)

    # Attendre que tous les threads se terminent
    for thread in threads:
        thread.join()
    
    print(f"%"*100)
    print("saved URLs : ",i+step)
    print(f"%"*100)
    
    time.sleep(5)

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
url_item = https://aujourdhui.ma/contact |||| by_thread_id = 1
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
url_item = https://lematin.ma/contact |||| by_thread_id = 2
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
url_item = https://www.goud.ma/%d9%84%d9%84%d8%a5%d8%aa%d8%b5%d8%a7%d9%84/ |||| by_thread_id = 3
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
url_item = https://lefaso.net/spip.php?article64819 |||| by_thread_id = 4
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
url_item = https://menara.ma/qui-sommes-nous |||| by_thread_id = 5
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
url_item = https://www.leconomiste.com/contact |||| by_thread_id = 6
%%%%%%%%%

In [7]:
shared_list.nb_URLs

101